### Задание 1

In [1]:
import pandas as pd

In [2]:
from sklearn.datasets import load_boston
boston = load_boston()

In [3]:
data = boston["data"]
feature_names = boston["feature_names"]
target = boston["target"]

X = pd.DataFrame(data, columns=feature_names)
y = pd.DataFrame(target, columns=["price"])

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [6]:
from sklearn.linear_model import LinearRegression

In [7]:
lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_test)

In [8]:
from sklearn.metrics import r2_score

In [9]:
r2_score(y_test, y_pred)

0.711226005748496

### Задание 2

In [10]:
from sklearn.ensemble import RandomForestRegressor

In [11]:
regr = RandomForestRegressor(n_estimators=1000, max_depth=12, random_state=42)
regr.fit(X_train, y_train.values[:, 0])
y_pred = regr.predict(X_test)
r2_score(y_test, y_pred)

0.87472606157312

In [12]:
# Для решения данной задачи модель, созданная с помощью метода RandomForestRegressor, работает лучше.